Patch Daddy: 

TO DO
    check how V_array is being sliced PRE_V ect
    outlier checks pre stats (AP width terrible)
    tau and sage for analysis extraction for comparison of like values i.e. same I injection - compare with similar voltage dif from resting
    build IV curves for pre and post drug aplication
    set up cache for figures as svg rather than pdf (should fix drug_aplication_visualisation() )
    
    plot pAD  - DJ what ecactly are the peak locations?

    handel if drug in/out is not specificed in AP file ... e.g control / put other data_type pAD to analise pAD from -100mV 
    change AP where it means Application to APP also in excel file !


    

DJ
    labeling of pAD
    plotters for pAD: including the display of averaged peaks with oe without each individual trace in low opacity 
    add steady_state_v to tau and sag lists [value, steady_state, I_injected, steady_state_v] 


In [1]:
#imports, load feature df, set directory and colours

# %load_ext autoreload
# %autoreload 2


from utils.mettabuild_functions import expandFeatureDF, loopCombinations_stats
# from utils.plotter import drug_aplication_visualisation, plot_all_FI_curves, plot_FI_AP_curves
from utils.base_utils import *
import openpyxl
import pandas as pd
import numpy as np
import os
import warnings

#CONSTANTS
ROOT = os.getcwd() #This gives terminal location (terminal working dir)
INPUT_DIR = f'{ROOT}/input'
OUTPUT_DIR = f'{ROOT}/output'
CACHE_DIR = f'{ROOT}/cache'

filename = "feature_df_py.xlsx"  # df of files and factors

color_dict = {"pAD":"orange","Somatic":"blue","WASH":"lightsteelblue", "PRE":"black", "CONTROL": 'grey', "TCB2":'green', "DMT":"teal", "PSIL":"orange", "LSD":"purple", "MDL":'blue', 'I_display':'cornflowerblue'} 
saveColors(filename, color_dict)




COLORS {'pAD': 'orange', 'Somatic': 'blue', 'WASH': 'lightsteelblue', 'PRE': 'black', 'CONTROL': 'grey', 'TCB2': 'green', 'DMT': 'teal', 'PSIL': 'orange', 'LSD': 'purple', 'MDL': 'blue', 'I_display': 'cornflowerblue'} SAVED TO /Users/jasminebutler/Desktop/IGOR_phd/cache/feature_df_py SUBCACHE


In [2]:
getorBuildExpandedDF(filename, 'feature_df_expanded', expandFeatureDF, from_scratch=False)

GETTING "feature_df_expanded" FROM "feature_df_py" CACHE


,folder_file,cell_ID,data_type,drug,replication_no,application_order,drug_in,drug_out,I_set,R_series_Mohm,...,AP_width,PRE_pAD_AP_locs,AP_locs,voltage_threshold,max_firing,WASH_V,PRE_V,AP_height,PRE_Somatic_AP_locs,rheobased_threshold
0,JJB210406/t8,CTG210406a,FP,PRE,1,0,NaN,NaN,NaN,NaN,...,"[1.9, 217.3, 461.0, 655.6, 892.9, 1111.9, 1336...",NaN,NaN,"[-42.3225, -42.8175, -42.786564, -44.024063, -...",25.377226,NaN,NaN,"[88.85249, 89.656876, 88.82156, 90.05907, 90.2...",NaN,77.500000
1,JJB210406/t9,CTG210406a,FP,PRE,2,0,NaN,NaN,NaN,NaN,...,"[1.8, 222.6, 479.5, 721.0, 935.8, 1191.6, 1428...",NaN,NaN,"[-42.66281, -42.167812, -42.66281, -42.724686,...",21.391115,NaN,NaN,"[92.131874, 91.667816, 92.16281, 91.51312, 91....",NaN,98.222222
2,JJB210406/t10,CTG210406a,FP,PRE,3,0,NaN,NaN,NaN,NaN,...,"[1.9, 526.9, 948.8, 1.8, 163.6, 320.1, 489.4, ...",NaN,NaN,"[-41.301563, -41.672813, -41.765625, -39.69281...",23.164274,NaN,NaN,"[92.78156, 92.8125, 92.41031, 89.19281, 90.739...",NaN,110.476190
3,JJB210406/t11,CTG210406a,AP,DMT,1,1,11.0,20.0,holding_I,NaN,...,NaN,[],"[139149, 278246, 129235, 55804, 392274, 377297...",NaN,NaN,[],"[[-76.07531, -55.40906, -56.399063, -56.1825, ...",NaN,[],NaN
4,JJB210406/t16,CTG210406a,FP,DMT,1,1,NaN,NaN,NaN,NaN,...,"[1.5, 283.2, 603.5, 933.7, 1350.7]",NaN,NaN,"[-39.3525, -38.115, -37.805626, -38.950314, -3...",9.025382,NaN,NaN,"[81.08719, 79.973434, 79.07625, 80.80875, 79.9...",NaN,50.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,JJB230524/t35,TLX230524c,FP,PRE,3,0,NaN,NaN,NaN,31.5,...,"[1.9, 301.9, 722.0, 1212.2, 1720.0]",NaN,NaN,"[-37.74375, -37.310623, -37.836563, -38.548126...",28.418199,NaN,NaN,"[85.635, 84.92343, 85.635, 86.22281, 86.09906]",NaN,131.818182
973,JJB230524/t36,TLX230524c,AP,TCB2,1,1,20.0,36.0,short step,NaN,...,NaN,[],"[10274, 214075, 214225]",NaN,NaN,[],"[[-75.36375, -73.69312, -73.445625, -73.07437,...",NaN,[],NaN
974,JJB230524/t37,TLX230524c,FP,TCB2,1,1,NaN,NaN,NaN,41.0,...,"[1.9, 246.0, 527.2, 812.9, 1080.8, 1324.3, 156...",NaN,NaN,"[-36.25875, -36.320625, -36.135, -36.10406, -3...",30.879565,NaN,NaN,"[79.91156, 80.12813, 79.91156, 80.00438, 80.46...",NaN,96.250000
975,JJB230524/t38,TLX230524c,FP,TCB2,2,1,NaN,NaN,NaN,41.0,...,"[2.1, 175.2, 534.5, 811.9, 1079.0, 1343.6, 161...",NaN,NaN,"[-38.115, -37.99125, -38.053123, -37.341564, -...",28.312136,NaN,NaN,"[75.735, 76.29187, 77.00343, 76.01344, 76.7559...",NaN,104.444444


In [3]:
#Initial Trace Plotting

#not working as run out of active memeory change pdf style to .svg and cashe 
# drug_aplication_visualisation(feature_df, {OUTPUT_DIR}, color_dict) # generates PDF of drug aplications # in plotters in utils

# plot_all_FI_curves(feature_df,  color_dict)  # generates PDF with all FI curves for single cell labed with drug and aplication order #### MAKE HZ NOT APs per sweep also isnt it in pA not nA??

# plot_FI_AP_curves(feature_df) #generated PDF with FI-AP for each cell

getColors(filename)


{'PRE': 'black',
 'CONTROL': 'grey',
 'TCB2': 'green',
 'DMT': 'teal',
 'PSIL': 'orange',
 'LSD': 'purple',
 'MDL': 'blue'}

In [3]:
#this function is dependant on values from feature_df_expanded
#this will generate all plots that could also be gotten by getorbuildHistogram(finename, identifier) #FIX ME
loopCombinations_stats(filename)
#RuntimeWarning: Mean of empty slice. #FIX ME

GETTING "feature_df_expanded" FROM "feature_df_py" CACHE


/Users/jasminebutler/Desktop/IGOR_phd/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jasminebutler/Desktop/IGOR_phd/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jasminebutler/Desktop/IGOR_phd/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jasminebutler/Desktop/IGOR_phd/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jasminebutler/Desktop/IGOR_phd/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jasminebutler/Desktop/IGOR_phd/.v

Insufficient data for L5a_CBLforFiring_(Hz)
Insufficient data for L5a_CBLforVoltage_Threshold_(mV)
Insufficient data for L5a_CBLfor AP_Height_(mV)
Insufficient data for L5a_CBLforAP_slope_(V_s^-1)
Insufficient data for L5a_CBLforAP_width_(s) 
Insufficient data for L5a_CBLforAP_latency_(ms)
p-value annotation legend:
      ns: p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

PRE vs. LSD: Custom statistical test, P_val:1.144e-02
SAVED /Users/jasminebutler/Desktop/IGOR_phd/output/Histogram/L5a_TLX_Firing_(Hz).svg
SAVED /Users/jasminebutler/Desktop/IGOR_phd/output/Histogram/L5a_TLX_Firing_(Hz).png
p-value annotation legend:
      ns: p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

PRE vs. LSD: Custom statistical test, P_val:1.385e-02
SAVED /Users/jasminebutler/Desktop/IGOR_phd/output/Histogram/L5a_TLX_

/Users/jasminebutler/Desktop/IGOR_phd/.venv/lib/python3.8/site-packages/seaborn/categorical.py:1402: UserWarning: You passed a edgecolor/edgecolors ('#4c4c4c') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  points = ax.scatter(cat_pos, swarm_data, s=s, **kws)


SAVED /Users/jasminebutler/Desktop/IGOR_phd/output/Histogram/L5a_TLX_pAD_vs_somatic_APs_LSD.svg
SAVED /Users/jasminebutler/Desktop/IGOR_phd/output/Histogram/L5a_TLX_pAD_vs_somatic_APs_LSD.png
Insuficient data for L5a_TLX with MDL application 
Insuficient data for L5a_TLX with PSIL application 


...

/Users/jasminebutler/Desktop/IGOR_phd/.venv/lib/python3.8/site-packages/seaborn/categorical.py:1402: UserWarning: You passed a edgecolor/edgecolors ('#4c4c4c') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  points = ax.scatter(cat_pos, swarm_data, s=s, **kws)


SAVED /Users/jasminebutler/Desktop/IGOR_phd/output/Histogram/L6b_DRD__pAD_vs_somatic_APs_MDL.svg
SAVED /Users/jasminebutler/Desktop/IGOR_phd/output/Histogram/L6b_DRD__pAD_vs_somatic_APs_MDL.png
Insuficient data for L6b_DRD_ with PSIL application 


/Users/jasminebutler/Desktop/IGOR_phd/.venv/lib/python3.8/site-packages/seaborn/categorical.py:1402: UserWarning: You passed a edgecolor/edgecolors ('#4c4c4c') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  points = ax.scatter(cat_pos, swarm_data, s=s, **kws)


SAVED /Users/jasminebutler/Desktop/IGOR_phd/output/Histogram/L6b_DRD__pAD_vs_somatic_APs_TCB2.svg
SAVED /Users/jasminebutler/Desktop/IGOR_phd/output/Histogram/L6b_DRD__pAD_vs_somatic_APs_TCB2.png


,folder_file,cell_ID,data_type,drug,replication_no,application_order,drug_in,drug_out,I_set,R_series_Mohm,...,mean_AP_slope_file,mean_AP_width_file,mean_AP_latency_file,max_firing_cell_drug,voltage_threshold_cell_drug,AP_height_cell_drug,AP_slope_cell_drug,AP_width_cell_drug,AP_latency_cell_drug,first_drug_AP
907,JJB230509/t11,CBL230509b,FP,PRE,1,0,NaN,NaN,NaN,47.0,...,82.629766,704.375000,0.975000,20.688653,-35.720416,78.239833,86.914814,778.40119,0.904365,TCB2
908,JJB230509/t12,CBL230509b,FP,PRE,2,0,NaN,NaN,NaN,44.0,...,90.947657,841.000000,0.866667,20.688653,-35.720416,78.239833,86.914814,778.40119,0.904365,TCB2
909,JJB230509/t13,CBL230509b,FP,PRE,3,0,NaN,NaN,NaN,44.0,...,87.167019,789.828571,0.871429,20.688653,-35.720416,78.239833,86.914814,778.40119,0.904365,TCB2
906,JJB230509/t8,CBL230509a,AP,TCB2,1,1,21.0,35.0,short step,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
910,JJB230509/t15,CBL230509b,AP,TCB2,1,1,9.0,22.0,short step,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,JJB230126/t42,DRD230126c,FP_AP,WASH,11,1,8.0,NaN,NaN,NaN,...,44.853426,767.400000,1.370000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
443,JJB230126/t43,DRD230126c,FP_AP,WASH,12,1,10.0,NaN,NaN,NaN,...,45.551643,685.200000,1.375000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
725,JJB230301/t19,DRD230301a,FP_AP,WASH,14,1,NaN,NaN,NaN,NaN,...,65.450391,843.366667,1.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
726,JJB230301/t20,DRD230301a,FP_AP,WASH,15,1,NaN,NaN,NaN,NaN,...,63.155565,884.650000,1.175000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
